In [26]:
%load_ext autoreload
%autoreload 1

import glob
import numpy as np
import pandas as pd
from collections import OrderedDict

import matplotlib.pyplot as plt
%matplotlib widget

# processed_directory = './data/processed_mats/*.mat'
processed_directory = '/Volumes/AnxietyBU/callbacks/processed/*.mat'

acceptable_call_labels = ['Call', 'Stimulus']  # any stimulus_trials containing call types NOT in this list are excluded (this includes unlabeled, which are stored as 'USV'!!)

files = [f for f in glob.glob(processed_directory)]

# or only specific files. Note: may mess up histograms, which may require data from >1 file
# files = [
#     './data/processed_mats/or60rd49-d1-20240425115050-Block1-PROCESSED.mat',
#     './data/processed_mats/or60rd49-d2-20240426114923-Block1-PROCESSED.mat'
# ]
files


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


['/Volumes/AnxietyBU/callbacks/processed/gr18bu69-d1-20240418115824-Block1-PROCESSED.mat',
 '/Volumes/AnxietyBU/callbacks/processed/gr18bu69-d1-20240418120341-Block5-PROCESSED.mat',
 '/Volumes/AnxietyBU/callbacks/processed/gr18bu69-d1-20240418120857-Block9-PROCESSED.mat',
 '/Volumes/AnxietyBU/callbacks/processed/gr18bu69-d2-20240419122320-Block0-PROCESSED.mat',
 '/Volumes/AnxietyBU/callbacks/processed/gr18bu69-d2-20240419122439-Block1-PROCESSED.mat',
 '/Volumes/AnxietyBU/callbacks/processed/gr18bu69-d2-20240419122558-Block2-PROCESSED.mat',
 '/Volumes/AnxietyBU/callbacks/processed/gr18bu69-d2-20240419122717-Block3-PROCESSED.mat',
 '/Volumes/AnxietyBU/callbacks/processed/gr18bu69-d2-20240419122836-Block4-PROCESSED.mat',
 '/Volumes/AnxietyBU/callbacks/processed/gr18bu69-d2-20240419122956-Block5-PROCESSED.mat',
 '/Volumes/AnxietyBU/callbacks/processed/gr18bu69-d2-20240419123115-Block6-PROCESSED.mat',
 '/Volumes/AnxietyBU/callbacks/processed/gr18bu69-d2-20240419123234-Block7-PROCESSED.mat',

In [27]:
%aimport utils.deepsqueak
from utils.deepsqueak import call_mat_stim_trial_loader, multi_index_from_dict

df = pd.DataFrame()

call_types_all = pd.DataFrame()
rejected_trials_all = pd.DataFrame()

for file in files:
    calls_df, stim_trials, rejected_trials, file_info, call_types = call_mat_stim_trial_loader(file, acceptable_call_labels=['Call', 'Stimulus'], verbose=False)

    # TODO: make this a nicely editable parameter
    multi_index_info = OrderedDict()
    multi_index_info['birdname'] = file_info['birdname']
    multi_index_info['day'] = int(file_info['d'])
    multi_index_info['block'] = int(file_info['block'])

    # create multiindex: birdname, stim_trial_index, call_index
    stim_trials = multi_index_from_dict(
        stim_trials, 
        multi_index_info, 
        keep_current_index=True,
    )
    df = pd.concat((df, stim_trials), axis='rows')
    
    rejected_trials = multi_index_from_dict(
        rejected_trials, 
        multi_index_info, 
        keep_current_index=True
    )
    rejected_trials_all = pd.concat((rejected_trials_all, rejected_trials), axis='rows')

    call_types = multi_index_from_dict(
        call_types, 
        multi_index_info, 
        keep_current_index=True
    )
    call_types_all = pd.concat((call_types_all, call_types), axis='rows')

print('Rejected trials:')
rejected_trials_all

Rejected trials:


trial_start_s  trial_end_s  stim_duration_s  \
birdname day block calls_index                                                
gr18bu69 2   1     1                 4.423258     6.982666         0.088230   
             3     1                 5.023158     6.784143         0.124337   
             4     1                 4.401429     6.308222         0.125853   
             5     1                 4.659141     7.238099         0.109070   
             6     1                 3.420354     4.999537         0.141016   
...                                       ...          ...              ...   
pk81rd39 2   5     4                 3.433295     5.735821         0.132006   
             6     7                 4.544741     6.382694         0.122577   
             7     5                 3.424621     5.815517         0.123625   
             8     5                 3.782561     5.731621         0.127816   
             9     5                 4.053357     5.903617         0.124673   

                               calls_in_range  
birdname day block calls_index                 
gr18bu69 2   1     1                      [0]  
             3     1                      [0]  
             4     1                      [0]  
             5     1                      [0]  
             6     1                      [0]  
...                                       ...  
pk81rd39 2   5     4             [3, 5, 6, 7]  
             6     7                   [6, 8]  
             7     5                      [4]  
             8     5                [4, 6, 7]  
             9     5                [4, 6, 7]  

[93 rows x 4 columns]

In [28]:
print(
    "Call types in rejected trials."
    + "\nLabel `USV` means an accepted call was not given a label."
    + "\nGo back to DeepSqueak & fix ths."
)

rejected_trial_call_types = call_types_all.loc[rejected_trials_all.index]
rejected_trial_call_types
# TODO: add stim index to rej trial type df (is this the first stim?)

# # see only blocks with a specific call type
#
# label = 'USV'
# label = 'Noise'
# call_types_all.loc[~np.isnan(call_types_all.loc[:, label])]

Call types in rejected trials.
Label `USV` means an accepted call was not given a label.
Go back to DeepSqueak & fix ths.


type                            Call  Loom_wing_flap  Song
birdname day block calls_index                            
gr18bu69 2   1     1             NaN             1.0   NaN
             3     1             NaN             1.0   NaN
             4     1             NaN             1.0   NaN
             5     1             NaN             1.0   NaN
             6     1             NaN             1.0   NaN
...                              ...             ...   ...
pk81rd39 2   5     4             3.0             1.0   NaN
             6     7             1.0             1.0   NaN
             7     5             NaN             1.0   NaN
             8     5             2.0             1.0   NaN
             9     5             2.0             1.0   NaN

[93 rows x 3 columns]

In [ ]:
# eliminate all block 0s - account for first loom bug

# raise Exception('Make sure you want to do this! You will need to reload the data afterward if you want block 0 back.')

# blocks = df.index.get_level_values(2)
# df = df[blocks != 0]

# df

In [29]:
df

trial_start_s  trial_end_s  stim_duration_s  \
birdname day block stims_index                                                
gr18bu69 1   1     0                 5.099209     7.973140         0.140258   
                   1                 7.973140    10.949204         0.144049   
                   2                10.949204    12.708204         0.150872   
                   3                12.708204    14.387013         0.155989   
                   4                14.387013    16.528022         0.147839   
...                                       ...          ...              ...   
pk81rd39 2   9     9                25.148495    28.034870         0.155216   
                   10               28.034870    29.240696         0.143030   
                   11               29.240696    32.124331         0.138435   
                   12               32.124331    33.327919         0.144656   
                   13               33.327919    34.086893         0.146674   

                                      calls_in_range  \
birdname day block stims_index                         
gr18bu69 1   1     0                 [5, 6, 7, 8, 9]   
                   1            [11, 12, 13, 14, 15]   
                   2            [17, 18, 19, 20, 21]   
                   3                [23, 24, 25, 26]   
                   4                [28, 29, 30, 31]   
...                                              ...   
pk81rd39 2   9     9                    [40, 41, 42]   
                   10                       [44, 45]   
                   11               [47, 48, 49, 50]   
                   12                           [52]   
                   13                           [54]   

                                                          call_times_stim_aligned  \
birdname day block stims_index                                                      
gr18bu69 1   1     0            [[0.3392721758908257, 0.4105382865807421], [0....   
                   1            [[0.20849128127369276, 0.2941622441243368], [0...   
                   2            [[0.20432145564821802, 0.2786201667930257], [0...   
                   3            [[0.16518195602729335, 0.24327141774071315], [...   
                   4            [[0.1884003032600461, 0.25966641394996337], [0...   
...                                                                           ...   
pk81rd39 2   9     9            [[0.6688919067382812, 0.7150363922119141], [1....   
                   10           [[0.20705448010053473, 0.29404993263369406], [...   
                   11           [[0.43816566467285156, 0.4801158905029297], [0...   
                   12                 [[0.3499603556237858, 0.39561373304483993]]   
                   13                [[0.17052530346179395, 0.25333187219703746]]   

                                n_calls  latency_s  \
birdname day block stims_index                       
gr18bu69 1   1     0                  5   0.339272   
                   1                  5   0.208491   
                   2                  5   0.204321   
                   3                  4   0.165182   
                   4                  4   0.188400   
...                                 ...        ...   
pk81rd39 2   9     9                  3   0.668892   
                   10                 2   0.207054   
                   11                 4   0.438166   
                   12                 1   0.349960   
                   13                 1   0.170525   

                                                                     wav_filename  \
birdname day block stims_index                                                      
gr18bu69 1   1     0            /Volumes/AnxietyBU/callbacks/gr18bu69/d1-basel...   
                   1            /Volumes/AnxietyBU/callbacks/gr18bu69/d1-basel...   
                   2            /Volumes/AnxietyBU/callbacks/gr18bu69/d1-basel...   
                   3            /Volumes/

In [30]:
all_birds = list(set(df.index.get_level_values(0)))
all_birds

['or60rd49',
 'pk81rd39',
 'pk63rd15',
 'gr18bu69',
 'or54rd45',
 'gr3bu36',
 'or14pu27']

## Plot Rasters

### Plotting Parameters

In [31]:
day_colors = {1: "#a2cffe", 2: "#840000"}
day_labels = {1: "baseline", 2: "loom"}

stim_kwargs = dict(alpha=0.5)
call_kwargs = dict(color="black", alpha=0.5)

### Raster by block

In [ ]:
%%capture  
# %%capture prevents plot output

%aimport utils.plot
from utils.plot import plot_callback_raster

save_folder =  None
save_folder = './data/figures/callback_rasters_by_block'

# every bird/day/block
unique_conditions = list(set([a[0:3] for a in df.index]))

## or select a subset
# unique_conditions = [
#     ('or14pu27', 1, 1),
#     ('or14pu27', 2, 1),
#     ('or54rd45', 1, 1),
#     ('or54rd45', 2, 1),
# ]

# figs = {}

for bird, day, block in unique_conditions:

    fig = plt.figure()
    ax = fig.subplots()

    data = df.loc[(bird, day, block)]
    
    title_str = f'{bird}-d{day}-b{block}'

    stim_kwargs['color'] = day_colors[day]

    plot_callback_raster(
        data,
        ax=ax,
        title = title_str,
        plot_stim_blocks = False,
        show_legend = True,
        call_kwargs = call_kwargs,
        stim_kwargs = stim_kwargs,
    )

    ax.set_xlim([-0.1, 3])

    # figs[title_str] = fig

    if save_folder is not None:
        fig.savefig(f'{save_folder}/{title_str}.png')

### Raster by day

In [ ]:
%%capture  
# %%capture prevents plot output

%aimport utils.plot
from utils.plot import plot_callback_raster_multiblock

save_folder =  None
save_folder = './data/figures/callback_rasters_multiblock'

# every bird/day
unique_conditions = list(set([a[0:2] for a in df.index]))

for bird, day in unique_conditions:
    data = df.loc[(bird, day)]

    title_str = f"{bird}-d{day}"

    stim_kwargs = dict(color=day_colors[day], alpha=0.5, edgecolor=None)
    call_kwargs = dict(color="black", alpha=0.5, edgecolor=None)

    fig = plt.figure()
    ax = fig.subplots()

    plot_callback_raster_multiblock(
        data,
        ax=ax,
        plot_hlines=True,
        show_block_axis=True,
        show_legend=False,
        xlim=[-0.1, 3],
        stim_kwargs = stim_kwargs,
        call_kwargs = call_kwargs,
        title = title_str,
    )

    if save_folder is not None:
        fig.savefig(f'{save_folder}/{title_str}.png')

## Violin plots

In [ ]:
# NOTE: violin plot only works for days = [1,2] right now

days = [1, 2]
width = 0.75

In [ ]:
%%capture  
# %%capture prevents plot output

%aimport utils.plot
from utils.plot import plot_violins_by_block

save_folder =  None
save_folder = './data/figures/n_calls'
# save_folder = './data/figures/n_calls-norm'

for bird in all_birds:
    fig, ax = plt.subplots()
    title_str = bird

    ax = plot_violins_by_block(
            df.loc[bird],
            field="n_calls",
            ax=ax,
            days=days,
            day_colors=day_colors,
            width=width,
            dropna=False,
    )

    ax.set(
        xlim=[-0.5,9.5],
        xticks= np.arange(0,10),
        # ylim=[-.5, 8],
        xlabel='Block',
        ylabel='Calls per stimulus',
        title=title_str,
    )
    
    if save_folder is not None:
        fig.savefig(f'{save_folder}/{title_str}.png')

In [ ]:
%%capture  
# %%capture prevents plot output

%aimport utils.plot
from utils.plot import plot_violins_by_block

save_folder =  None
save_folder = './data/figures/latency'
# save_folder = './data/figures/latency-norm'

for bird in all_birds:
    fig, ax = plt.subplots()
    title_str = bird

    ax = plot_violins_by_block(
            df.loc[bird],
            field="latency_s",
            ax=ax,
            days=days,
            day_colors=day_colors,
            width=width,
            dropna=True,
    )

    ax.set(
        xlim=[-0.5,9.5],
        xticks= np.arange(0,10),
        # ylim=[0, 2.5],
        xlabel='Block',
        ylabel='Latency to first call (s)',
        title=title_str,
    )
    
    if save_folder is not None:
        fig.savefig(f'{save_folder}/{title_str}.png')

## Histograms

All blocks merged

### Latency

In [36]:
%%capture

# index levels: 'birdname', 'day', 'block', 'stims_index'
# idx = pd.IndexSlice
# this_bird = df.loc[idx[birdname, :, :, :]]

save_folder = None
save_folder = './data/figures/histograms/latency'

%aimport utils.plot
from utils.plot import plot_group_hist

for bird in all_birds:

    fig, ax = plt.subplots()

    plot_group_hist(
        df.loc[bird],
        field="latency_s",
        grouping_level="day",
        group_colors=day_colors,
        alt_labels={1: "baseline", 2: "loom"},
        ax=ax,
        density=True,
        ignore_nan=True,
        histogram_kwargs={
            "range": (0, 1.5),
            "bins": 40,
        },
        stair_kwargs={
            1: {"hatch": "/"},
            2: {"hatch": "\\"},
        },
    )

    ax.set(
        title=f"{bird}: latency to first call",
        xlabel="Latency (s)",
    )

    if save_folder is not None:
        fig.savefig(f'{save_folder}/{bird}-latency.png')

In [37]:
%%capture

save_folder = None
save_folder = "./data/figures/histograms/n_calls"
# save_folder = './data/figures'

%aimport utils.plot
from utils.plot import plot_group_hist

for bird in all_birds:

    fig, ax = plt.subplots()

    plot_group_hist(
        df.loc[bird],
        field="n_calls",
        grouping_level="day",
        group_colors=day_colors,
        alt_labels={1: "baseline", 2: "loom"},
        ax=ax,
        density=True,
        ignore_nan=False,
        histogram_kwargs={
            "range": (-0.5, 9.5),
            "bins": 10,
        },
        stair_kwargs={
            1: {"hatch": "/"},
            2: {"hatch": "\\"},
        },
    )

    ax.set(
        title=f"{bird}: number of calls per trial",
        xlabel="# of calls",
        xticks=list(range(0, 10)),
    )

    if save_folder is not None:
        fig.savefig(f"{save_folder}/{bird}-ncalls.png")